In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-21T00:00:00Z,01001,"11.05.2021, 00:00 Uhr",0,-9,2021-03-19T00:00:00Z,0,1,1,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-21T00:00:00Z,01001,"11.05.2021, 00:00 Uhr",0,-9,2021-03-21T00:00:00Z,0,1,0,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-22T00:00:00Z,01001,"11.05.2021, 00:00 Uhr",0,-9,2021-03-22T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-23T00:00:00Z,01001,"11.05.2021, 00:00 Uhr",0,-9,2021-03-19T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-23T00:00:00Z,01001,"11.05.2021, 00:00 Uhr",0,-9,2021-03-22T00:00:00Z,0,1,1,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-21,90164.0,...,1,0,0,0,0,1,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-21,90164.0,...,1,0,0,0,0,1,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-22,90164.0,...,1,0,0,0,0,1,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-23,90164.0,...,1,0,0,0,0,1,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-23,90164.0,...,1,0,0,0,0,1,0,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
454,2021-05-06,17193,20,8656,8337,200,695,2023,5287,6499,2160,501,28,109227.0,15603.857143,0.87,83166711,131.34,Deutschland,Nation
455,2021-05-07,15306,11,7729,7423,154,634,1844,4748,5722,1928,421,9,105355.0,15050.714286,0.86,83166711,126.68,Deutschland,Nation
456,2021-05-08,11832,5,5817,5872,143,600,1498,3539,4417,1459,304,15,102121.0,14588.714286,0.87,83166711,122.79,Deutschland,Nation
457,2021-05-09,5125,4,2604,2453,68,246,583,1648,1878,603,160,7,100623.0,14374.714286,0.89,83166711,120.99,Deutschland,Nation
458,2021-05-10,4333,3,2214,2049,70,161,474,1392,1583,558,152,13,95808.0,13686.857143,0.88,83166711,115.20,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
6841,Thüringen,2021-05-06,16,665,2133378,1,323,341,1,13,...,38,0,655.75,618.000000,1.027016,0.91,4326.0,202.78,Thüringen,Bundesland
6842,Thüringen,2021-05-07,16,595,2133378,2,268,326,1,16,...,29,0,709.25,579.857143,1.361324,0.86,4059.0,190.26,Thüringen,Bundesland
6843,Thüringen,2021-05-08,16,449,2133378,0,216,231,2,17,...,17,0,620.75,567.714286,1.226173,0.86,3974.0,186.28,Thüringen,Bundesland
6844,Thüringen,2021-05-09,16,211,2133378,0,114,96,1,7,...,11,0,480.00,554.000000,0.847682,0.86,3878.0,181.78,Thüringen,Bundesland
6845,Thüringen,2021-05-10,16,236,2133378,0,123,111,2,7,...,13,1,372.75,533.285714,0.568433,0.86,3733.0,174.98,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
133316,StadtRegion Aachen,2021-05-06,05334,Nordrhein-Westfalen,81,557026.0,0,39,41,1,...,6,0,112.25,105.428571,1.025114,0.82,738.0,132.49,StadtRegion Aachen,Landkreis
133317,StadtRegion Aachen,2021-05-07,05334,Nordrhein-Westfalen,113,557026.0,0,52,61,0,...,3,0,127.50,98.142857,1.495601,0.77,687.0,123.33,StadtRegion Aachen,Landkreis
133318,StadtRegion Aachen,2021-05-08,05334,Nordrhein-Westfalen,54,557026.0,0,27,27,0,...,3,0,101.25,94.000000,1.205357,0.78,658.0,118.13,StadtRegion Aachen,Landkreis
133319,StadtRegion Aachen,2021-05-09,05334,Nordrhein-Westfalen,26,557026.0,0,12,14,0,...,2,0,68.50,91.714286,0.668293,0.80,642.0,115.25,StadtRegion Aachen,Landkreis
133320,StadtRegion Aachen,2021-05-10,05334,Nordrhein-Westfalen,30,557026.0,0,19,11,0,...,0,0,55.75,88.571429,0.496659,0.84,620.0,111.31,StadtRegion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
133316,2021-05-06,81,0,39,41,1,3,8,18,33,...,557026.0,132.49,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,112.25,1.025114,StadtRegion Aachen,05334
133317,2021-05-07,113,0,52,61,0,6,7,50,36,...,557026.0,123.33,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,127.50,1.495601,StadtRegion Aachen,05334
133318,2021-05-08,54,0,27,27,0,1,3,23,16,...,557026.0,118.13,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,101.25,1.205357,StadtRegion Aachen,05334
133319,2021-05-09,26,0,12,14,0,0,1,8,10,...,557026.0,115.25,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,68.50,0.668293,StadtRegion Aachen,05334
133320,2021-05-10,30,0,19,11,0,0,0,17,12,...,557026.0,111.31,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,55.75,0.496659,StadtRegion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')